# YOLOv8 Installation

In [2]:
# Create YOLOv8 root folder
!mkdir yolov8-architecture

mkdir: cannot create directory 'yolov8-architecture': File exists


In [3]:
# Go to YOLOv8 root folder
%cd yolov8-architecture

/kaggle/working/yolov8-architecture


In [4]:
# Install YOLOv8
!pip install ultralytics==8.0.196

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6277.6/8062.4 GB disk)


In [5]:
# Download the YOLOv8 Architecture File
!wget https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/models/v8/yolov8.yaml

--2025-04-24 20:01:20--  https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/cfg/models/v8/yolov8.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1977 (1.9K) [text/plain]
Saving to: 'yolov8.yaml'

yolov8.yaml         100%[===================>]   1.93K  --.-KB/s    in 0s      

2025-04-24 20:01:20 (27.2 MB/s) - 'yolov8.yaml' saved [1977/1977]



# YOLOv8 Architecture

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture.png" height="750"/>
</div>

In [6]:
# Create YOLOv8l Architecture
!cp yolov8.yaml yolov8m.yaml

## Modified YOLOv8 Architecture for Small Objects

In [10]:
# Copy YOLOv8l Small Architecture
!cp yolov8.yaml yolov8l-small.yaml

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-modification-for-small-object.png" height="685"/>
</div>

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-for-small-object.png" height="692"/>
</div>

### Training a Modified Model (Small Objects)

In [ ]:
# Training a Modified Model (Small Objects)
!yolo detect train model=yolov8l-small.yaml data=data/sperm.yaml workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8_sperm_small

## Modified YOLOv8 Architecture for Medium Objects

In [7]:
# Copy YOLOv8l Medium Architecture
!cp yolov8.yaml yolov8m-medium.yaml

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-modification-for-medium-object.png" height="685"/>
</div>

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-for-medium-object.png" height="685"/>
</div>

### Training a Modified Model (Medium Objects)

In [9]:
%%writefile /kaggle/working/yolov8-architecture/yolov8m-medium.yaml
# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv8 object detection model with P3-P5 outputs. For Usage examples see https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80 # number of classes
#scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  #n: [0.33, 0.25, 1024] # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  #s: [0.33, 0.50, 1024] # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
 # m: [0.67, 0.75, 768] # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
 # l: [1.00, 1.00, 512] # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  #x: [1.00, 1.25, 512] # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

depth_multiple: 0.67
width_multiple: 0.75
max_channels: 768
# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  #- [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  #- [[-1, 4], 1, Concat, [1]] # cat backbone P3
  #- [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [4, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium) -> 15

 # - [-1, 1, Conv, [512, 3, 2]]
  #- [[-1, 9], 1, Concat, [1]] # cat head P5
 # - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15], 1, Detect, [nc]] # Detect(P3, P4, P5)

Overwriting /kaggle/working/yolov8-architecture/yolov8m-medium.yaml


In [10]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="GKR5d1Hy1bW1WBLrIg0Z")
project = rf.workspace("phone-1nwp8").project("phone-detect-xqya5")
version = project.version(36)
dataset = version.download("yolov8")
                

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 88.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires j


Extracting Dataset Version Zip to Phone-detect-36 in yolov8:: 100%|██████████| 18888/18888 [00:02<00:00, 7096.45it/s]


In [11]:
import yaml

# Path to your data.yaml file
yaml_path = '/kaggle/working/yolov8-architecture/Phone-detect-36/data.yaml'

# Load the yaml file
with open(yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)

# Print the current configuration to see what needs to be changed
print(data_config)


{'names': ['0'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'phone-detect-xqya5', 'url': 'https://universe.roboflow.com/phone-1nwp8/phone-detect-xqya5/dataset/36', 'version': 36, 'workspace': 'phone-1nwp8'}, 'test': 'test/images', 'train': 'train/images', 'val': 'valid/images'}


In [12]:
# Modify the paths to the train and validation images
data_config['train'] = '/kaggle/working/yolov8-architecture/Phone-detect-36/train/images'
data_config['val'] = '/kaggle/working/yolov8-architecture/Phone-detect-36/valid/images'
data_config['test'] = '/kaggle/working/yolov8-architecture/Phone-detect-36/test/images'

# If there are other paths that need to be updated, modify them here as well
# e.g., data_config['test'] = 'path/to/test/images'

# Print the updated configuration to verify changes
print(data_config)


{'names': ['0'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'phone-detect-xqya5', 'url': 'https://universe.roboflow.com/phone-1nwp8/phone-detect-xqya5/dataset/36', 'version': 36, 'workspace': 'phone-1nwp8'}, 'test': '/kaggle/working/yolov8-architecture/Phone-detect-36/test/images', 'train': '/kaggle/working/yolov8-architecture/Phone-detect-36/train/images', 'val': '/kaggle/working/yolov8-architecture/Phone-detect-36/valid/images'}


In [13]:
# Save the updated yaml file
with open(yaml_path, 'w') as file:
    yaml.safe_dump(data_config, file)

print(f"Updated data.yaml successfully!")


Updated data.yaml successfully!


In [14]:
# Training a Modified Model (Medium Objects)
!yolo task=detect mode=train model=/kaggle/working/yolov8-architecture/yolov8m-medium.yaml data=/kaggle/working/yolov8-architecture/Phone-detect-36/data.yaml  batch=-1 device=0 epochs=40 patience=50 name=yolov8_phone


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   2655744  ultralytics.nn.modules.conv.Conv             [384

In [10]:
# Training a Modified Model (Medium Objects)
!yolo task=detect mode=train model=yolov8m.pt data=/kaggle/working/pandas-2-9/data.yaml  batch=-1 device=0 epochs=40 patience=50 name=yolov8_septembersa

New https://pypi.org/project/ultralytics/8.3.23 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/working/pandas-2-9/data.yaml, epochs=40, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolov8_septembersa, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualiz

## Modified YOLOv8 Architecture for Big Objects

In [ ]:
# Copy YOLOv8l Big Architecture
!cp yolov8.yaml yolov8l-big.yaml

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-modification-for-big-object.png" height="685"/>
</div>

<div>
  <img src="https://www.stunningvisionai.com/course/yolov8-architecture-for-big-object.png" height="720"/>
</div>

### Download the dataset

In [ ]:
# Go to the data folder
%cd data

In [ ]:
# Download the dataset
!gdown https://drive.google.com/uc?id=1FW0QR0jITbhLiHL_B5HqYrowQtoQukFX

In [ ]:
# Unzip Dataset
!unzip tomato_leaf_dataset.zip

### Download the data file

In [ ]:
# Download the data file
!gdown https://drive.google.com/uc?id=1Q50HdUtbLscs03I2HFm5r4Ah0gWHS5Kd

In [ ]:
# Go to the root folder
%cd ../

### Training

In [ ]:
# Training Original Model
!yolo detect train model=yolov8l.yaml data=data/tomato_leaf.yaml workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8_tomato_leaf

### Training a Modified Model (Big Objects)

# YOLOv8 for big objects with TensorRT

In [15]:
!pip install tensorrt==8.6.1 tensorrt_lean==8.6.1 tensorrt_dispatch==8.6.1

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16971 sha256=f900a78aa3c570c53e3658528eb22ff7c521c1fadbf5c4a8567333aa586b51a2
  Stored in directory: /root/.cache/pip/wheels/6d/29/56/abdffd4c604f255b5254bef3f1c598ab7811ea020540599438
  Created wheel for tensorrt_lean: filename=tensorrt_lean-8.6.1-py2.py3-none-any.whl size=17053 sha256=17a36a7e4b3aeb019c181057786c2cb1b7feee190f7dfcff0939aa19793d695f
  Stored in directory: /root/.cache/pip/wheels/c9/e7/d5/6397d8f9cb33a20787497416f27ac8db4f847c6ab810f28717
  Created wheel for tensorrt_dispatch: filename=tensorrt_dispatch-8.6.1-py2.py3-none-any.whl size=17124 sha256=e8c4e887d92f7c9afaf4a04b48c60069295be4d585a348803aef298d91a45c0d
  Stored in directory: /root/.cache/pip/wheels/44/57/7a/546a598e72b22868df0166b1951ffb0b1ad6d69c9d3ce962cd
Successfully built tensorrt tensorrt_lean tensorrt_dis

In [17]:
!pip install onnx==1.14.1 onnxsim==0.4.33 onnxruntime-gpu==1.16.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 75.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:0000:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 MB 2.2 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: onnx
    Found existing installation: onnx 1.16.1
    Uninstalling onnx-1.16.1:
      Successfully uninstalled onnx-1.16.1


## Convert the Model


In [18]:
# Export YOLOv8 Model to Tensorrt
!yolo export model=/kaggle/working/yolov8-architecture/runs/detect/yolov8_inveep2/weights/best.pt format=engine half=True device=0

Ultralytics YOLOv8.0.196 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8m-medium summary (fused): 197 layers, 53132640 parameters, 0 gradients, 149.8 GFLOPs

PyTorch: starting from '/kaggle/working/yolov8-architecture/runs/detect/yolov8_inveep2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 20, 1600) (101.6 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.33...
ONNX: export success ✅ 5.1s, saved as '/kaggle/working/yolov8-architecture/runs/detect/yolov8_inveep2/weights/best.onnx' (101.4 MB)

TensorRT: starting export with TensorRT 8.6.1...
[07/14/2024-23:25:31] [TRT] [I] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 718, GPU 1010 (MiB)
[07/14/2024-23:25:37] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +248, GPU +48, now: CPU 1042, GPU 1058 (MiB)
[07/14/2024-23:25:37] [TRT] [I] ----------------------------------------------------------------
[07/14/2024-23:25:37] 

In [15]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [16]:
download_file('/kaggle/working/yolov8-architecture/runs/detect/yolov8_phone/weights/best.pt', 'phone')


/kaggle/working/phone.zip